### [Analysis] Mortality estimation: linear risk model

#### This set of demonstrative code shows how to estimate the ambient O${_3}$ exposure-associated mortality using linear (log-linear) risk model. 

In [1]:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

#### Ambient O${_3}$ dataset is developed in global scale with 1/8°×1/8° spatial resolution, and thus we have to select the grids in China territory. 

In [2]:
data = pd.read_csv('ozone_17_mort_est_total(1119).csv')
data = data[data['arcgis_ID'] == 44]

#### Here we prescribe all necessary parameters: multi-cause exposure-response RRs with 95% CIs, and the low-concentration cut-off (LCC, or threshold). The pooled RRs are estimated by meta-analysis (see a "tutorial" code file). The LCC value(s) are defined as the lowest 5${^{th}}$ percentile of the multi-data fused ambient ozone dataset (only considering the residential land areas) following Malley et al. 2017. 

In [3]:
lnRR_allcause = np.log(1.016)
lnRR_sigma_allcause = (np.log(1.021) - np.log(1.016))/1.96 

lnRR_ncd = np.log(1.016)
lnRR_sigma_ncd = (np.log(1.021) - np.log(1.016))/1.96 

lnRR_resp = np.log(1.042)
lnRR_sigma_resp = (np.log(1.055) - np.log(1.042))/1.96 

lnRR_copd = np.log(1.060) 
lnRR_sigma_copd = (np.log(1.080) - np.log(1.060))/1.96 

lnRR_cvd = np.log(1.017)
lnRR_sigma_cvd = (np.log(1.024) - np.log(1.017))/1.96 

lnRR_ihd = np.log(1.024)
lnRR_sigma_ihd = (np.log(1.040) - np.log(1.024))/1.96 

LCC = 44.5

#### For the sake of quick computation (e.g. less than 10 minutes on JASMIN supercomputation system) as a demonstration, we have simplified some trivial aspects: (1) we cluster the age ≥25 population rather than 5-yr stratification. (2) we do not calculate the arithmetic mean and standard deviations, as we choose to report the median with 95% uncertainty intervals. 

In [4]:
start = time.process_time()

for t in range(0,61607,1):
    
    ## Extract the age ≥25 risked proportion 
    pop_all_age       = data.iloc[t,25]
    pop_all_age_sigma = data.iloc[t,26]
    pop_25_age        = data.iloc[t,27]
    pop_25_age_sigma  = data.iloc[t,28]
    
    ## Extract the baseline mortality rate 
    allcause_25_mort       = data.iloc[t,29]
    allcause_25_mort_sigma = data.iloc[t,30]
    resp_25_mort           = data.iloc[t,33]
    resp_25_mort_sigma     = data.iloc[t,34]
    copd_25_mort           = data.iloc[t,37]
    copd_25_mort_sigma     = data.iloc[t,38]
    cvd_25_mort            = data.iloc[t,41]
    cvd_25_mort_sigma      = data.iloc[t,42]
    ihd_25_mort            = data.iloc[t,45]
    ihd_25_mort_sigma      = data.iloc[t,46]
    ncd_25_mort            = data.iloc[t,49]
    ncd_25_mort_sigma      = data.iloc[t,50]
    
    ## Extract urban and rural population and MDA8h-ozone exposure 
    pop_urban_grid = data.iloc[t,10] 
    o3_mda8h_urban = data.iloc[t,22] 
    pop_rural_grid = data.iloc[t,11] 
    o3_mda8h_rural = data.iloc[t,23] 

    ## Convert into the hazadous dose from exposure concentration
    o3_mda8h_urban_dose = max(0,o3_mda8h_urban - LCC) 
    o3_mda8h_rural_dose = max(0,o3_mda8h_rural - LCC) 
    
    ## Generate Gaussian distributed samples 
    pop_all_age_dist      = np.random.normal(pop_all_age, pop_all_age_sigma, 1000) 
    pop_25_age_dist       = np.random.normal(pop_25_age , pop_25_age_sigma , 1000)
    
    allcause_25_mort_dist = np.random.normal(allcause_25_mort, allcause_25_mort_sigma, 1000)
    resp_25_mort_dist     = np.random.normal(resp_25_mort    , resp_25_mort_sigma    , 1000)
    copd_25_mort_dist     = np.random.normal(copd_25_mort    , copd_25_mort_sigma    , 1000)
    cvd_25_mort_dist      = np.random.normal(cvd_25_mort     , cvd_25_mort_sigma     , 1000)
    ihd_25_mort_dist      = np.random.normal(ihd_25_mort     , ihd_25_mort_sigma     , 1000)
    ncd_25_mort_dist      = np.random.normal(ncd_25_mort     , ncd_25_mort_sigma     , 1000)
    
    RR_dist_allcause = np.random.normal(lnRR_allcause, lnRR_sigma_allcause, 1000)
    RR_dist_resp     = np.random.normal(lnRR_resp    , lnRR_sigma_resp    , 1000)
    RR_dist_copd     = np.random.normal(lnRR_copd    , lnRR_sigma_copd    , 1000)
    RR_dist_cvd      = np.random.normal(lnRR_cvd     , lnRR_sigma_cvd     , 1000)
    RR_dist_ihd      = np.random.normal(lnRR_ihd     , lnRR_sigma_ihd     , 1000)
    RR_dist_ncd      = np.random.normal(lnRR_ncd     , lnRR_sigma_ncd     , 1000)
    
    ## Start the Monte Carlo bootstrap simulation realisations 
    delta_mort_allcause_urban = allcause_25_mort_dist * (1 - np.exp(- RR_dist_allcause * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_allcause_rural = allcause_25_mort_dist * (1 - np.exp(- RR_dist_allcause * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    delta_mort_resp_urban     = resp_25_mort_dist     * (1 - np.exp(- RR_dist_resp     * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_resp_rural     = resp_25_mort_dist     * (1 - np.exp(- RR_dist_resp     * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    delta_mort_copd_urban     = copd_25_mort_dist     * (1 - np.exp(- RR_dist_copd     * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_copd_rural     = copd_25_mort_dist     * (1 - np.exp(- RR_dist_copd     * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    delta_mort_cvd_urban      = cvd_25_mort_dist      * (1 - np.exp(- RR_dist_cvd      * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_cvd_rural      = cvd_25_mort_dist      * (1 - np.exp(- RR_dist_cvd      * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    delta_mort_ihd_urban      = ihd_25_mort_dist      * (1 - np.exp(- RR_dist_ihd      * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_ihd_rural      = ihd_25_mort_dist      * (1 - np.exp(- RR_dist_ihd      * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    delta_mort_ncd_urban      = ncd_25_mort_dist      * (1 - np.exp(- RR_dist_ncd      * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## urban
    delta_mort_ncd_rural      = ncd_25_mort_dist      * (1 - np.exp(- RR_dist_ncd      * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 ## rural
    
    ## Summarise the results 
    est_allcause_urban = np.nanpercentile(delta_mort_allcause_urban,50) 
    lb_allcause_urban  = np.nanpercentile(delta_mort_allcause_urban,2.5)
    ub_allcause_urban  = np.nanpercentile(delta_mort_allcause_urban,97.5)

    est_allcause_rural = np.nanpercentile(delta_mort_allcause_rural,50) 
    lb_allcause_rural  = np.nanpercentile(delta_mort_allcause_rural,2.5)
    ub_allcause_rural  = np.nanpercentile(delta_mort_allcause_rural,97.5)

    est_resp_urban = np.nanpercentile(delta_mort_resp_urban,50) 
    lb_resp_urban  = np.nanpercentile(delta_mort_resp_urban,2.5)
    ub_resp_urban  = np.nanpercentile(delta_mort_resp_urban,97.5)

    est_resp_rural = np.nanpercentile(delta_mort_resp_rural,50) 
    lb_resp_rural  = np.nanpercentile(delta_mort_resp_rural,2.5)
    ub_resp_rural  = np.nanpercentile(delta_mort_resp_rural,97.5)
    
    est_copd_urban = np.nanpercentile(delta_mort_copd_urban,50) 
    lb_copd_urban  = np.nanpercentile(delta_mort_copd_urban,2.5)
    ub_copd_urban  = np.nanpercentile(delta_mort_copd_urban,97.5)

    est_copd_rural = np.nanpercentile(delta_mort_copd_rural,50) 
    lb_copd_rural  = np.nanpercentile(delta_mort_copd_rural,2.5)
    ub_copd_rural  = np.nanpercentile(delta_mort_copd_rural,97.5)
    
    est_cvd_urban = np.nanpercentile(delta_mort_cvd_urban,50) 
    lb_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,2.5)
    ub_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,97.5)

    est_cvd_rural = np.nanpercentile(delta_mort_cvd_rural,50) 
    lb_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,2.5)
    ub_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,97.5)
    
    est_ihd_urban = np.nanpercentile(delta_mort_ihd_urban,50) 
    lb_ihd_urban  = np.nanpercentile(delta_mort_ihd_urban,2.5)
    ub_ihd_urban  = np.nanpercentile(delta_mort_ihd_urban,97.5)
    
    est_ihd_rural = np.nanpercentile(delta_mort_ihd_rural,50) 
    lb_ihd_rural  = np.nanpercentile(delta_mort_ihd_rural,2.5)
    ub_ihd_rural  = np.nanpercentile(delta_mort_ihd_rural,97.5)
    
    est_ncd_urban = np.nanpercentile(delta_mort_ncd_urban,50) 
    lb_ncd_urban  = np.nanpercentile(delta_mort_ncd_urban,2.5)
    ub_ncd_urban  = np.nanpercentile(delta_mort_ncd_urban,97.5)

    est_ncd_rural = np.nanpercentile(delta_mort_ncd_rural,50) 
    lb_ncd_rural  = np.nanpercentile(delta_mort_ncd_rural,2.5)
    ub_ncd_rural  = np.nanpercentile(delta_mort_ncd_rural,97.5)
    
    ## Store the distribution statistics into the DataFrame 
    data.iloc[t,53]  = est_allcause_urban
    data.iloc[t,54]  =  lb_allcause_urban
    data.iloc[t,55]  =  ub_allcause_urban

    data.iloc[t,58]  = est_allcause_rural
    data.iloc[t,59]  =  lb_allcause_rural
    data.iloc[t,60]  =  ub_allcause_rural
    
    data.iloc[t,63]  = est_resp_urban
    data.iloc[t,64]  =  lb_resp_urban
    data.iloc[t,65]  =  ub_resp_urban

    data.iloc[t,68]  = est_resp_rural
    data.iloc[t,69]  =  lb_resp_rural
    data.iloc[t,70]  =  ub_resp_rural
    
    data.iloc[t,73]  = est_copd_urban
    data.iloc[t,74]  =  lb_copd_urban
    data.iloc[t,75]  =  ub_copd_urban

    data.iloc[t,78]  = est_copd_rural
    data.iloc[t,79]  =  lb_copd_rural
    data.iloc[t,80]  =  ub_copd_rural
    
    data.iloc[t,83]  = est_cvd_urban
    data.iloc[t,84]  =  lb_cvd_urban
    data.iloc[t,85]  =  ub_cvd_urban

    data.iloc[t,88]  = est_cvd_rural
    data.iloc[t,89]  =  lb_cvd_rural
    data.iloc[t,90]  =  ub_cvd_rural
    
    data.iloc[t,93]  = est_ihd_urban
    data.iloc[t,94]  =  lb_ihd_urban
    data.iloc[t,95]  =  ub_ihd_urban

    data.iloc[t,98]  = est_ihd_rural
    data.iloc[t,99]  =  lb_ihd_rural
    data.iloc[t,100] =  ub_ihd_rural
    
    data.iloc[t,103]  = est_ncd_urban
    data.iloc[t,104]  =  lb_ncd_urban
    data.iloc[t,105]  =  ub_ncd_urban

    data.iloc[t,108]  = est_ncd_rural
    data.iloc[t,109]  =  lb_ncd_rural
    data.iloc[t,110] =  ub_ncd_rural
    
    if t%1000 == 0:
        print(t) ## Add a tracer to check the simulation progresses 
    
elapsed = (time.process_time() - start)
print('Time used:', elapsed, "s")    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
Time used: 520.911625742 s


In [5]:
data.to_csv('china_ozone_mort_2019(1112).csv', index = False)